In [3]:
##%matplotlib inline
import csv
import nltk
#from nltk.stem.porter import PorterStemmer
from stem import IndonesianStemmer
import string
#import os
import re
from collections import defaultdict
import multiprocessing
import dill
import utils

In [4]:
#stopwords = nltk.corpus.stopwords.words('indonesian')

In [2]:
stemmer = IndonesianStemmer()
tokenizer = nltk.tokenize.RegexpTokenizer("[\w']+", flags=re.UNICODE)

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemm er.stem(item))
    return stemmed

def tokenize(text):
    global stemmer, tokenizer
    #tokens = nltk.word_tokenize(text, language="indonesian")
    tokens = tokenizer.tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [4]:
%%time
token_dict = defaultdict(lambda: None)
product_category = defaultdict(lambda: None)

with open('/data/python-scrapy-data-mining/corpus.csv') as raw_file:
    for (prod_id, title, url) in csv.reader(raw_file):

        url = url.split('/')
        if url[3] != 'p':
            continue
        cat = '/'.join(url[4:-1])
        product_category[int(prod_id)] = cat
        token_dict[int(prod_id)] = tokenize(
            title.lower().translate(None, string.punctuation))

CPU times: user 13 s, sys: 320 ms, total: 13.3 s
Wall time: 13.5 s


In [5]:
%time collection = nltk.text.TextCollection(token_dict.values())

CPU times: user 712 ms, sys: 124 ms, total: 836 ms
Wall time: 852 ms


In [7]:
#get a list of unique terms
print "Toal products:", len(token_dict)
unique_terms = list(set(collection))
print "Unique terms found: ", len(unique_terms)

Toal products: 228562
Unique terms found:  101633


In [ ]:
# Function to create a TF*IDF vector for one document.  For each of
# our unique words, we have a feature which is the td*idf for that word
# in the current document
def TFIDF(document):
    global collection
    document_tfidf = {}
    for word in unique_terms:
        if word not in document:
            continue
        ifidf = collection.tf_idf(word, document)
        if ifidf > 0.01:
            document_tfidf[word] = ifidf
    return document_tfidf

In [12]:
TFIDF(token_dict[1])

Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 233, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 267, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1044, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1004, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 491, in getmodule
    if ismodule(module) and hasatt

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [22]:
%%time

def term_idf(terms):
    global collection
    return [(t, collection.idf(t)) for t in terms]

pool = multiprocessing.Pool(processes=8)
terms = pool.map(term_idf, utils.chunks(unique_terms, 8))

CPU times: user 2.89 s, sys: 1.99 s, total: 4.88 s
Wall time: 25min 49s


In [30]:
import operator
#%time terms = reduce(operator.add, terms, [])
terms = sorted(terms, key=lambda t: -t[1])
print "Total unique terms:", len(terms)

Total unique terms: 101633


In [34]:
# Inverted Document Frequencies (of the unique terms):
terms[:10] + ["..."] + terms[-10:]

[('fawn', 12.339562787303892),
 ('tsukino', 12.339562787303892),
 ('5quot', 12.339562787303892),
 ('400410', 12.339562787303892),
 ('daiich', 12.339562787303892),
 ('asustrixgtx750tioc2gd5', 12.339562787303892),
 ('lodyne', 12.339562787303892),
 ('inlite', 12.339562787303892),
 ('n7100note', 12.339562787303892),
 ('ds1015', 12.339562787303892),
 '...',
 ('sepatu', 3.674984608974596),
 ('usb', 3.6517832953121205),
 ('samsung', 3.642551169311818),
 ('black', 3.6282841721734576),
 ('1', 3.6197553358259373),
 ('case', 3.552342458674594),
 ('jam', 3.5344875434332073),
 ('for', 3.494073550550618),
 ('tas', 3.3190520876119853),
 ('original', 3.1732788015246287)]

In [44]:
%time backup_object(terms, "terms.pkl")

CPU times: user 1.99 s, sys: 328 ms, total: 2.32 s
Wall time: 4.93 s


In [73]:
%time len([TFIDF(token_dict[k]) for k in token_dict.keys()[1000:2000]])

CPU times: user 2min 9s, sys: 844 ms, total: 2min 10s
Wall time: 2min 9s


1000

In [120]:
# Vectorize products (into a list of tuples (product_id, vector representation)):
%time vectors = [(prod_id,TFIDF(token_dict[prod_id])) for prod_id in token_dict.keys()]

CPU times: user 8h 18min 35s, sys: 5min 16s, total: 8h 23min 52s
Wall time: 8h 18min 47s


In [191]:
vectors[1], vectors[22]

((2,
  defaultdict(<function __main__.<lambda>>,
              {'4171': 0.49787583755381826,
               'and': 0.22686251165346813,
               'arrival': 0.45189397793206915,
               'box': 0.2233729500125791,
               'case': 0.1614701117579361,
               'erika': 0.4663691475283662,
               'feminino': 0.5109522953925355,
               'frame': 0.25059170744305903,
               'gafas': 0.5293825275792703,
               'glasses': 0.7510932130361975,
               'kacamata': 0.2328236070730618,
               'keren': 0.2166144446332587,
               'logo': 0.2868765575942361,
               'masculino': 0.5293825275792703,
               'men': 0.2954055059059315,
               'new': 0.19716081677671735,
               'sun': 0.351111012293756,
               'sunglasses': 0.3563524144078921,
               'velvet': 0.3563524144078921,
               'with': 0.20376618155491144,
               'women': 0.30641549219570535})),
 (23,
  defa

In [123]:
dill.dump(vectors, open("vectors.pkl", "wb"))
#dill.dump(token_dict, open("token_dict.pkl", "wb"))
#dill.dump(terms, open("terms.pkl", "wb"))
#dill.dump(collection, open("collection.pkl", "wb"))

In [92]:
import math

In [192]:
def norm(vect):
    if type(vect) == tuple:
        vect = vect[1]
    return math.sqrt(sum((v * v for v in vect.values())))
def distance(v1, v2):
    if type(v1) == tuple:
        v1 = v1[1]
        v2 = v2[1] 
    dist = sum((v1[w] * v2[w] for w in set(v1).intersection(v2)), 0.0)
    return dist / (norm(v1) * norm(v2))

In [193]:
%%time 
for i in xrange(len(vectors)):
    vectors[i] =  vectors[i][0], vectors[i][1], norm(vectors[i])

CPU times: user 1.06 s, sys: 4.24 s, total: 5.3 s
Wall time: 50 s


In [195]:
dill.dump(vectors, open("vectors.pkl", "wb"))

In [194]:
def norm(vect):
    if type(vect) == tuple:
        vect = vect[1]
    return math.sqrt(sum((v * v for v in vect.values())))
def distance(v1, v2):
    v1, n1 = v1[1:3]
    v2, n2 = v2[1:3] 
    dist = sum((v1[w] * v2[w] for w in set(v1).intersection(v2)), 0.0)
    return dist / (n1 * n1)

In [196]:
vectors[1]

(2,
 defaultdict(<function __main__.<lambda>>,
             {'4171': 0.49787583755381826,
              'and': 0.22686251165346813,
              'arrival': 0.45189397793206915,
              'box': 0.2233729500125791,
              'case': 0.1614701117579361,
              'erika': 0.4663691475283662,
              'feminino': 0.5109522953925355,
              'frame': 0.25059170744305903,
              'gafas': 0.5293825275792703,
              'glasses': 0.7510932130361975,
              'kacamata': 0.2328236070730618,
              'keren': 0.2166144446332587,
              'logo': 0.2868765575942361,
              'masculino': 0.5293825275792703,
              'men': 0.2954055059059315,
              'new': 0.19716081677671735,
              'sun': 0.351111012293756,
              'sunglasses': 0.3563524144078921,
              'velvet': 0.3563524144078921,
              'with': 0.20376618155491144,
              'women': 0.30641549219570535}),
 1.7515000162534715)

In [188]:
#sum( (vectors[1][1][w]*vectors[1][1][w] for w in  set(vectors[1][1]).intersection(vectors[2][1])))
#vectors[4], vectors[214], norm(vectors[4]), norm(vectors[214]), 
##distance(vectors[2], vectors[1])
#prod_ids = sorted([prod_id for prod_id, _ in vectors])
#prod_ids[:10]
(len(vectors) / 2) * 2.41 / (60*60)

76.50478055555556

In [201]:
%%time
sim = []
start = 13000 
end = start + 10000
for i in xrange(start, end):
    for j in xrange(i + 1, len(vectors)):
        try: 
            dist = distance(vectors[i], vectors[j])
            if dist > 0.01:
                sim.append((vectors[i][0], vectors[j][0], round(dist, 5)))
        except Exception as e:
            print e
            print "***", i, j

KeyboardInterrupt: 

In [197]:
import csv
with open("sim_{0}_{1}.csv".format(start, end), "w") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(sim)

In [ ]:
#this can take some time
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='indonesian')
tfs = tfidf.fit_transform(token_dict.values())

In [205]:
dir()

['In',
 'IndonesianStemmer',
 'Out',
 'TFIDF',
 '_',
 '_105',
 '_106',
 '_107',
 '_108',
 '_110',
 '_111',
 '_112',
 '_113',
 '_114',
 '_115',
 '_116',
 '_118',
 '_121',
 '_122',
 '_124',
 '_126',
 '_128',
 '_129',
 '_130',
 '_131',
 '_132',
 '_134',
 '_135',
 '_136',
 '_140',
 '_144',
 '_145',
 '_146',
 '_150',
 '_151',
 '_153',
 '_154',
 '_157',
 '_16',
 '_162',
 '_165',
 '_166',
 '_17',
 '_172',
 '_173',
 '_174',
 '_175',
 '_177',
 '_18',
 '_184',
 '_185',
 '_186',
 '_187',
 '_188',
 '_19',
 '_191',
 '_196',
 '_198',
 '_199',
 '_203',
 '_204',
 '_22',
 '_24',
 '_25',
 '_26',
 '_28',
 '_29',
 '_30',
 '_31',
 '_32',
 '_36',
 '_37',
 '_39',
 '_40',
 '_41',
 '_42',
 '_44',
 '_45',
 '_51',
 '_52',
 '_53',
 '_55',
 '_56',
 '_58',
 '_59',
 '_6',
 '_60',
 '_61',
 '_62',
 '_64',
 '_65',
 '_66',
 '_67',
 '_7',
 '_70',
 '_73',
 '_74',
 '_75',
 '_76',
 '_77',
 '_81',
 '_83',
 '_9',
 '_97',
 '_99',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__name__',
 '__package__',
 '_dh',
 

In [208]:
del()